In [1]:
# | results: hide
import pandas as pd
from datetime import datetime
import requests
import holoviews as hv
import locale

locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")
hv.extension("bokeh", logo=False)

In [2]:
VAT = 1.19
YEAR = 2025
TADO_FIXED_FEE_PER_KWH = 0.21540  # EUR/kWh
AEUW_FEE_PER_KWH = 0.3462  # EUR/kWh
BASE_FEE_AUEW = 15.68

In [3]:
all_interval = pd.read_csv("../data/2025_messwerte_pseudonymisiert_3600s_interval.csv", parse_dates=["time"])

In [4]:
all_interval

,name,time,Wh
0,Wohnung 2,2025-01-01 00:00:00,410.0
1,Wohnung 2,2025-01-01 01:00:00,381.0
2,Wohnung 2,2025-01-01 02:00:00,258.0
3,Wohnung 2,2025-01-01 03:00:00,220.0
4,Wohnung 2,2025-01-01 04:00:00,278.0
...,...,...,...
85297,Sunny_Tripower_Gesamtertrag,2025-12-22 07:00:00,235.0
85298,Sunny_Tripower_Gesamtertrag,2025-12-22 08:00:00,1195.0
85299,Sunny_Tripower_Gesamtertrag,2025-12-22 09:00:00,2056.0
85300,Sunny_Tripower_Gesamtertrag,2025-12-22 10:00:00,2550.0


In [5]:
consumption = all_interval.loc[lambda x: x["name"].isin(["Wohnung 1", "Wohnung 2", "Wohnung 3", "Wohnung 4"])]
production = all_interval.loc[
    lambda x: x["name"].isin(
        ["Sunny_Island_Batterie_entladen", "Sunny_Island_Netzbezug", "Sunny_Tripower_Gesamtertrag"]
    )
]

In [6]:
data = consumption.groupby("name")["Wh"].sum().astype(int) / 1000
hv.Bars(data).opts(ylabel="kWh", width=600, height=400)

:Bars   [name]   (Wh)

In [7]:
production_relative = (
    production.groupby("time")
    .apply(lambda x: x.assign(frac=x["Wh"] / x["Wh"].sum()))
    .reset_index(drop=True)
    .drop(columns=["Wh"])
    .rename(columns={"name": "source"})
)

/tmp/ipykernel_509448/2937385568.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(frac=x["Wh"] / x["Wh"].sum()))


In [8]:
production_relative

,source,time,frac
0,Sunny_Island_Batterie_entladen,2025-01-01 00:00:00,0.000000
1,Sunny_Island_Netzbezug,2025-01-01 00:00:00,1.000000
2,Sunny_Tripower_Gesamtertrag,2025-01-01 00:00:00,0.000000
3,Sunny_Island_Batterie_entladen,2025-01-01 01:00:00,0.000000
4,Sunny_Island_Netzbezug,2025-01-01 01:00:00,1.000000
...,...,...,...
25588,Sunny_Tripower_Gesamtertrag,2025-12-22 09:00:00,0.990366
25589,Sunny_Island_Batterie_entladen,2025-12-22 10:00:00,0.000000
25590,Sunny_Island_Netzbezug,2025-12-22 10:00:00,0.007782
25591,Sunny_Tripower_Gesamtertrag,2025-12-22 10:00:00,0.992218


In [8]:
# Get hourly prices from awattar API
begin = datetime.fromisoformat("2020-01-01").timestamp()
end = datetime.now().timestamp()
# API endpoint URL
url = f"https://api.awattar.de/v1/marketdata?start={int(begin) * 1000}&end={int(end) * 1000}"

# Fetch JSON data from the API
response = requests.get(url)
data = response.json()

# Convert JSON data to Pandas DataFrame
hourly_price = pd.DataFrame(data["data"])

hourly_price["start"] = pd.to_datetime(hourly_price["start_timestamp"], unit="ms")
hourly_price["end"] = pd.to_datetime(hourly_price["end_timestamp"], unit="ms")
hourly_price["marketprice"], hourly_price["unit"] = hourly_price["marketprice"] / 1000, "EUR/kWh"
hourly_price["start_day"] = hourly_price["start"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)
hourly_price["real_price"] = hourly_price["marketprice"] * VAT + TADO_FIXED_FEE_PER_KWH

In [13]:
production_relative

,source,time,frac
0,Sunny_Island_Batterie_entladen,2025-01-01 00:00:00,0.000000
1,Sunny_Island_Netzbezug,2025-01-01 00:00:00,1.000000
2,Sunny_Tripower_Gesamtertrag,2025-01-01 00:00:00,0.000000
3,Sunny_Island_Batterie_entladen,2025-01-01 01:00:00,0.000000
4,Sunny_Island_Netzbezug,2025-01-01 01:00:00,1.000000
...,...,...,...
25588,Sunny_Tripower_Gesamtertrag,2025-12-22 09:00:00,0.990366
25589,Sunny_Island_Batterie_entladen,2025-12-22 10:00:00,0.000000
25590,Sunny_Island_Netzbezug,2025-12-22 10:00:00,0.007782
25591,Sunny_Tripower_Gesamtertrag,2025-12-22 10:00:00,0.992218


In [9]:
hourly_price

,start_timestamp,end_timestamp,marketprice,unit,start,end,start_day,real_price
0,1577833200000,1577836800000,0.04188,EUR/kWh,2019-12-31 23:00:00,2020-01-01 00:00:00,12-31 23:00:00,0.265237
1,1577836800000,1577840400000,0.03860,EUR/kWh,2020-01-01 00:00:00,2020-01-01 01:00:00,01-01 00:00:00,0.261334
2,1577840400000,1577844000000,0.03655,EUR/kWh,2020-01-01 01:00:00,2020-01-01 02:00:00,01-01 01:00:00,0.258895
3,1577844000000,1577847600000,0.03232,EUR/kWh,2020-01-01 02:00:00,2020-01-01 03:00:00,01-01 02:00:00,0.253861
4,1577847600000,1577851200000,0.03085,EUR/kWh,2020-01-01 03:00:00,2020-01-01 04:00:00,01-01 03:00:00,0.252111
...,...,...,...,...,...,...,...,...
52596,1767178800000,1767182400000,0.08305,EUR/kWh,2025-12-31 11:00:00,2025-12-31 12:00:00,12-31 11:00:00,0.314229
52597,1767182400000,1767186000000,0.08517,EUR/kWh,2025-12-31 12:00:00,2025-12-31 13:00:00,12-31 12:00:00,0.316752
52598,1767186000000,1767189600000,0.08518,EUR/kWh,2025-12-31 13:00:00,2025-12-31 14:00:00,12-31 13:00:00,0.316764
52599,1767189600000,1767193200000,0.09083,EUR/kWh,2025-12-31 14:00:00,2025-12-31 15:00:00,12-31 14:00:00,0.323488


In [18]:
cost_df = (
    consumption.merge(production_relative, on="time")
    .merge(hourly_price, left_on="time", right_on="start")
    .loc[lambda x: x["source"] == "Sunny_Island_Netzbezug"]
    .assign(Wh_grid = lambda x: x["Wh"] * x["frac"])
    .assign(
        price_tado=lambda x: x["Wh_grid"] * x["real_price"] / 1000, price_auew=lambda x: x["Wh_grid"] * AEUW_FEE_PER_KWH / 1000
    )
)

In [19]:
cost_df

,name,time,Wh,source,frac,start_timestamp,end_timestamp,marketprice,unit,start,end,start_day,real_price,Wh_grid,price_tado,price_auew
1,Wohnung 2,2025-01-01 00:00:00,410.0,Sunny_Island_Netzbezug,1.000000,1735689600000,1735693200000,0.00160,EUR/kWh,2025-01-01 00:00:00,2025-01-01 01:00:00,01-01 00:00:00,0.217304,410.000000,0.089095,0.141942
4,Wohnung 2,2025-01-01 01:00:00,381.0,Sunny_Island_Netzbezug,1.000000,1735693200000,1735696800000,0.00000,EUR/kWh,2025-01-01 01:00:00,2025-01-01 02:00:00,01-01 01:00:00,0.215400,381.000000,0.082067,0.131902
7,Wohnung 2,2025-01-01 02:00:00,258.0,Sunny_Island_Netzbezug,1.000000,1735696800000,1735700400000,-0.00001,EUR/kWh,2025-01-01 02:00:00,2025-01-01 03:00:00,01-01 02:00:00,0.215388,258.000000,0.055570,0.089320
10,Wohnung 2,2025-01-01 03:00:00,220.0,Sunny_Island_Netzbezug,1.000000,1735700400000,1735704000000,-0.00001,EUR/kWh,2025-01-01 03:00:00,2025-01-01 04:00:00,01-01 03:00:00,0.215388,220.000000,0.047385,0.076164
13,Wohnung 2,2025-01-01 04:00:00,278.0,Sunny_Island_Netzbezug,1.000000,1735704000000,1735707600000,-0.00006,EUR/kWh,2025-01-01 04:00:00,2025-01-01 05:00:00,01-01 04:00:00,0.215329,278.000000,0.059861,0.096244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102354,Wohnung 4,2025-12-22 06:00:00,0.0,Sunny_Island_Netzbezug,0.020502,1766383200000,1766386800000,0.09045,EUR/kWh,2025-12-22 06:00:00,2025-12-22 07:00:00,12-22 06:00:00,0.323036,0.000000,0.000000,0.000000
102357,Wohnung 4,2025-12-22 07:00:00,0.0,Sunny_Island_Netzbezug,0.021122,1766386800000,1766390400000,0.09334,EUR/kWh,2025-12-22 07:00:00,2025-12-22 08:00:00,12-22 07:00:00,0.326475,0.000000,0.000000,0.000000
102360,Wohnung 4,2025-12-22 08:00:00,0.0,Sunny_Island_Netzbezug,0.000000,1766390400000,1766394000000,0.09079,EUR/kWh,2025-12-22 08:00:00,2025-12-22 09:00:00,12-22 08:00:00,0.323440,0.000000,0.000000,0.000000
102363,Wohnung 4,2025-12-22 09:00:00,0.0,Sunny_Island_Netzbezug,0.009634,1766394000000,1766397600000,0.08925,EUR/kWh,2025-12-22 09:00:00,2025-12-22 10:00:00,12-22 09:00:00,0.321607,0.000000,0.000000,0.000000


In [22]:
cost_df.groupby("name").apply(
    lambda x: x.set_index("time")[["Wh", "Wh_grid", "price_tado", "price_auew"]].resample("MS").sum()
).reset_index()

/tmp/ipykernel_509448/1955450972.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cost_df.groupby("name").apply(


,name,time,Wh,Wh_grid,price_tado,price_auew
0,Wohnung 1,2025-01-01,323575.0,157001.931737,55.862797,54.354069
1,Wohnung 1,2025-02-01,278893.0,67060.757801,26.106008,23.216434
2,Wohnung 1,2025-03-01,299192.0,12021.814608,4.187036,4.161952
3,Wohnung 1,2025-04-01,283628.0,3279.581531,1.098568,1.135391
4,Wohnung 1,2025-05-01,285928.0,1937.328291,0.644602,0.670703
5,Wohnung 1,2025-06-01,292621.0,4140.163941,1.361569,1.433325
6,Wohnung 1,2025-07-01,322396.0,4210.461795,1.420826,1.457662
7,Wohnung 1,2025-08-01,305798.0,6550.389136,2.167932,2.267745
8,Wohnung 1,2025-09-01,312732.0,42095.095084,14.258459,14.573322
9,Wohnung 1,2025-10-01,339789.0,42627.596393,13.646989,14.757674


In [23]:
cost_df.groupby("name")[["Wh", "Wh_grid", "price_tado", "price_auew"]].sum()

,Wh,Wh_grid,price_tado,price_auew
name,,,,
Wohnung 1,3637097.0,596880.671131,208.911458,206.640088
Wohnung 2,1730636.0,558260.489724,195.963817,193.269782
Wohnung 3,1973203.0,327568.271493,120.350094,113.404136
Wohnung 4,583843.0,78888.247279,28.821280,27.311111
